In [2]:
#######################
# LigthGBM Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l1/kfold' % DataBaseDir
OutputDir = '%s/l2/kfold' % DataBaseDir
kfold = 5
InputCols = ['lgb_l2', 'lgb_huber', 'lgb_fair', 'etr', 'en', 'knn', 'rf', 'xgb_rmse', 'lassolars', 
             'gbr_ls', 'gbr_huber', 'gbr_lad', 'rgf', 'rgf_sib', 'cb_ef']
#InputCols = ['lgb_l2', 'etr', 'knn']
strategy = 'lgb_fair'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
join_keys = ['air_store_id', 'visit_date']
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    for i in range(len(InputCols)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        if(i == 0):
            FoldValid = valid
            FoldHoldout = holdout
            FoldTest = test
        else:
            FoldValid = FoldValid.merge(valid[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)
            FoldHoldout = FoldHoldout.merge(holdout[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)
            FoldTest = FoldTest.merge(test[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)  
    FoldValid['fold'] = fold
    valid_dfs.append(FoldValid)
    holdout_dfs.append(FoldHoldout)
    test_dfs.append(FoldTest)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('----------------- CHECK -------------')
print('\n============================ TrainData')
print(TrainData.head(10))
print('\n============================ HoldoutData')
print(holdout_dfs[0].head(10))
print('\n============================ HoldoutData')
print(test_dfs[0].head(10))
print('-------------------------------------')
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
# parameters
params = {
    "boosting": "gbdt",
#     "objective": "regression_l1",
#     "lambda_l1": 500,  
#     "objective": "regression_l2",
#     "lambda_l2": 200,
    "objective": "fair",
    "fair_c": 10,
#     "objective": "huber",
#     "alpha": 2,
    
    "num_iterations": 100,
    "learning_rate": 0.06,
    "min_data_in_leaf": 100,
    'num_leaves': 127,
    "max_depth": 6,

    "feature_fraction": 0.9,
    "bagging_fraction": 0.85,
    "bagging_freq": 8,
    "min_hessian": 0.1,
    
    'max_bin': 255,
}
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    col = [c for c in FoldData['train'].columns if c not in ['id', 'air_store_id', 'visit_date', 'visitors', 'hpg_store_id', 'fold']]
    # train
    d_cv = lightgbm.Dataset(FoldData['train'][col], label= FoldData['train']['visitors'].values, max_bin= params['max_bin'], silent= True, free_raw_data= True)
    model = lightgbm.train(params, d_cv)
    # for valid
    FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
    rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
    cv_score += rmsle_valid
    # for holdout
    FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.predict(FoldData['test'][col])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l2/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
os.system('zip %s/%s.zip %s.csv' % (SubmitDir, OutputFileName, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

----------------- CHECK -------------

============================ TrainData
           air_store_id visit_date  visitors    lgb_l2  lgb_huber  lgb_fair  \
0  air_ba937bf13d40fb24 2016-01-15  3.401197  3.247568   3.433135  3.457238   
1  air_ba937bf13d40fb24 2016-01-29  3.295837  3.309194   3.381564  3.455909   
2  air_ba937bf13d40fb24 2016-01-30  1.945910  3.194028   3.198036  3.239608   
3  air_ba937bf13d40fb24 2016-02-10  3.496508  2.936297   3.065789  3.103343   
4  air_ba937bf13d40fb24 2016-02-13  2.197225  3.205479   3.177832  3.241215   
5  air_ba937bf13d40fb24 2016-02-17  2.890372  2.936297   3.065789  3.103343   
6  air_ba937bf13d40fb24 2016-02-24  3.091042  2.920160   3.065789  3.105565   
7  air_ba937bf13d40fb24 2016-03-09  3.178054  3.103057   3.125537  3.214320   
8  air_ba937bf13d40fb24 2016-03-27  0.693147  2.337237   2.162255  2.735153   
9  air_ba937bf13d40fb24 2016-03-30  3.258097  3.314060   3.290239  3.279237   

        etr        en       knn        rf  xgb_rmse 

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is 

fold 0: valid score 0.492884, holdout score 0.485316, valid length 45371
saving for 0th fold data done.
fold 1: valid score 0.484063, holdout score 0.485208, valid length 45371
saving for 1th fold data done.
fold 2: valid score 0.487636, holdout score 0.484566, valid length 45371
saving for 2th fold data done.
fold 3: valid score 0.492026, holdout score 0.484780, valid length 45370
saving for 3th fold data done.
fold 4: valid score 0.486533, holdout score 0.484889, valid length 45370
saving for 4th fold data done.

CV score 0.4886, Holdout score 0.4850, Elapsed time: 29.00s

